In [3]:
import torch
import torch.nn as nn
from torch.utils.data import random_split, TensorDataset
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

## Set de datos

antes de armar la red neuronal vamos a importar los datos que van a requerir usando pandas. vamos a hacer un formato sea compatible con la red reuronal que armaremos.

In [6]:
datos = pd.read_csv('Churn_Modelling.csv')
datos.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#separamos la ultima colmna para que se varible destino
datos_y = datos[datos.columns[-1]]
datos_y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [11]:
#Elminanos las columnas que no funcionaran
datos_x = datos.drop(columns=['RowNumber','CustomerId','Surname','Exited'])
datos_x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [13]:
#hacemos un one hot encoding, ya que las redes reuronales no procesan data de str
datos_x = pd.get_dummies(datos_x)
datos_x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0


## Escalamos datos

Ahora vamos a escalar los valores para que se encuentre dentro de un rango mas corto

In [14]:
escalador = StandardScaler()
datos_x = escalador.fit_transform(datos_x)
print(datos_x)

[[-0.32622142  0.29351742 -1.04175968 ... -0.57380915  1.09598752
  -1.09598752]
 [-0.44003595  0.19816383 -1.38753759 ...  1.74273971  1.09598752
  -1.09598752]
 [-1.53679418  0.29351742  1.03290776 ... -0.57380915  1.09598752
  -1.09598752]
 ...
 [ 0.60498839 -0.27860412  0.68712986 ... -0.57380915  1.09598752
  -1.09598752]
 [ 1.25683526  0.29351742 -0.69598177 ... -0.57380915 -0.91241915
   0.91241915]
 [ 1.46377078 -1.04143285 -0.35020386 ... -0.57380915  1.09598752
  -1.09598752]]


## Dividimos datos de entrenamiento y test

In [44]:
datos_x.shape #tengo 10 000 registros en 13 columnas

(10000, 13)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datos_x, datos_y, test_size=0.2,random_state=2)

In [46]:
print('X_train:{}, X_test:{}, y_train:{}, y_test:{}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train:(8000, 13), X_test:(2000, 13), y_train:(8000,), y_test:(2000,)


In [47]:
n_entradas = X_train.shape[1]
n_entradas

13

# Pytorch

In [81]:
#data in cpu

In [83]:
# t_X_train = torch.from_numpy(X_train).float().to('cpu')
# t_X_test = torch.from_numpy(X_test).float().to('cpu')

# t_y_train = torch.from_numpy(y_train.values).float().to('cpu')
# t_y_test = torch.from_numpy(y_test.values).float().to('cpu')

# t_y_train = t_y_train[:,None]
# t_y_test = t_y_test[:,None]

In [82]:
#data in GPU

In [85]:
t_X_train = torch.from_numpy(X_train).float().to('cuda')
t_X_test = torch.from_numpy(X_test).float().to('cuda')

t_y_train = torch.from_numpy(y_train.values).float().to('cuda')
t_y_test = torch.from_numpy(y_test.values).float().to('cuda')

t_y_train = t_y_train[:,None]
t_y_test = t_y_test[:,None]

In [86]:
test = TensorDataset(t_x_test,t_y_test)
print(test[0])

(tensor([ 0.8326, -0.0879,  1.7245,  0.7569,  0.8077,  0.6461, -1.0307,  1.2390,
         0.9972, -0.5787, -0.5738,  1.0960, -1.0960]), tensor([1.], device='cuda:0'))


## cosntruccion de red neuronal 

In [87]:
class Red(nn.Module):
    def __init__(self, n_entradas):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_entradas, 15)
        self.linear2 = nn.Linear(15, 8)
        self.linear3 = nn.Linear(8, 1)
    
    def forward(self, inputs):
        pred_1 = torch.sigmoid(input=self.linear1(inputs))
        pred_2 = torch.sigmoid(input=self.linear2(pred_1))
        pred_f = torch.sigmoid(input=self.linear3(pred_2))
        return pred_f

In [88]:
%%time
lr = 0.001
epochs = 2000
estatus_print = 100

model = Red(n_entradas=n_entradas).cuda() # si quieres entrenar con cuda sino lo quitas
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
print("Arquitectura del modelo: {}".format(model))

historico = pd.DataFrame()

print("Entranando el modelo")
for epoch in range(1, epochs+1):
    y_pred= model(t_X_train)
    loss = loss_fn(input=y_pred, target=t_y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % estatus_print == 0:
        print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")
    
    with torch.no_grad():
        y_pred = model(t_X_test)
        y_pred_class = y_pred.round()
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % estatus_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
    
    df_tmp = pd.DataFrame(data={
        'Epoch': epoch,
        'Loss': round(loss.item(), 4),
        'Accuracy': round(accuracy.item(), 4)
    }, index=[0])
    historico = pd.concat(objs=[historico, df_tmp], ignore_index=True, sort=False)

print("Accuracy final: {}".format(round(accuracy.item(), 4)))

Arquitectura del modelo: Red(
  (linear1): Linear(in_features=13, out_features=15, bias=True)
  (linear2): Linear(in_features=15, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=1, bias=True)
)
Entranando el modelo

Epoch 100 	 Loss: 0.5235
Accuracy: 80.85000610351562

Epoch 200 	 Loss: 0.4975
Accuracy: 80.85000610351562

Epoch 300 	 Loss: 0.4895
Accuracy: 80.85000610351562

Epoch 400 	 Loss: 0.4835
Accuracy: 80.85000610351562

Epoch 500 	 Loss: 0.4774
Accuracy: 80.85000610351562

Epoch 600 	 Loss: 0.4706
Accuracy: 80.85000610351562

Epoch 700 	 Loss: 0.4631
Accuracy: 80.85000610351562

Epoch 800 	 Loss: 0.455
Accuracy: 80.85000610351562

Epoch 900 	 Loss: 0.4468
Accuracy: 80.85000610351562

Epoch 1000 	 Loss: 0.4392
Accuracy: 80.85000610351562

Epoch 1100 	 Loss: 0.4326
Accuracy: 80.85000610351562

Epoch 1200 	 Loss: 0.4272
Accuracy: 81.30000305175781

Epoch 1300 	 Loss: 0.4227
Accuracy: 81.95000457763672

Epoch 1400 	 Loss: 0.4186
Accuracy: 82.4500045776367

cuda: 5.89
cpu: 7.33